# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[28334, 28335, 28336]

In [5]:
from Forecaster import Forecaster

In [6]:
forecasters = [Forecaster({id: IFP(id) for id in today_ids}) for i in range(5)]

In [7]:
for fcst in forecasters:
    fcst.fit()

MODEL DOMAIN
28334 Will there be a deadly attack within Israel causing at least five deaths before October 1, 2024?
Civil unrest
28335 Will any of the Waltons be worth $100 billion or more on September 30, 2024, according to Forbes?
Market cap
28336 Will Prince Tom Iseghohi win the September 21, 2024 Edo state gubernatorial election in Nigeria?
Election outcome
CORRELATOR
|Attack in Israel|28334|Will there be a deadly attack within Israel causing at least five deaths before October 1, 2024?|
|Net Worth of Waltons|28335|Will any of the Waltons be worth $100 billion or more on September 30, 2024, according to Forbes?|
|Edo State Election|28336|Will Prince Tom Iseghohi win the September 21, 2024 Edo state gubernatorial election in Nigeria?|
NEWS
ASKING
Researching Will any of the Waltons be worth $100 billion or more on September 30, 2024, according to Forbes?
ALL
A 'buy-and-hold' investor who invested $10,000 in Johnson & Johnson (JNJ) on September 20, 2004, would have seen their investm

In [8]:
for i, fcst in enumerate(forecasters):
    print("forecaster", i)
    fcst.report()
    print("=====================================================================")

forecaster 0
28334 Will there be a deadly attack within Israel causing at least five deaths before October 1, 2024?
Forecast 65
Rationale Given the recent attack on September 18, 2024, by Hezbollah in the Golan Heights, which resulted in the deadliest attack on Israeli civilians since the start of the Gaza War this month, and the ongoing high tensions in the region, it is plausible that another deadly attack causing at least five deaths could occur before October 1, 2024. The pattern of retaliatory attacks and the apparent escalation between Israel and its adversaries such as Hezbollah and Hamas indicates a high likelihood of another significant incident within this period. 

28335 Will any of the Waltons be worth $100 billion or more on September 30, 2024, according to Forbes?
Forecast 97
Rationale Based on the recent trends and the fact that both Jim and Rob Walton have already surpassed the $100 billion mark according to multiple sources, it is highly likely they will still be worth

In [9]:
import numpy as np
from Summarizer import Summarizer
from MetacGPT import MetacGPT

In [10]:
self = Forecaster({id: IFP(id) for id in today_ids})

In [11]:
editor = Summarizer(MetacGPT)

In [12]:
for ifp in self.ifps.values():
    ifp.forecast = np.mean([fcst.ifps[ifp.id].forecast for fcst in forecasters])
    ifp.rationale = editor.summarize('\n'.join([fcst.ifps[ifp.id].rationale for fcst in forecasters]))

In [13]:
self.report()

28334 Will there be a deadly attack within Israel causing at least five deaths before October 1, 2024?
Forecast 87.8
Rationale Considering the recent severe attack by Hezbollah on September 18, 2024, in the Golan Heights, along with the high and escalating tensions in the region, the consistent pattern of retaliatory violence, and credible reports of multiple deadly incidents, it is highly probable—approaching certainty—that another deadly attack causing at least five deaths will occur within Israel before October 1, 2024. 

28335 Will any of the Waltons be worth $100 billion or more on September 30, 2024, according to Forbes?
Forecast 94.2
Rationale Given that Jim and Rob Walton both currently have net worths exceeding $100 billion, with Alice Walton closely approaching that threshold, and considering the consistent upward trend in Walmart's stock performance, it is highly probable that at least one member of the Walton family will have a net worth of $100 billion or more on September

In [14]:
self.upload()

Prediction posted for  28334
Comment posted for  28334
Prediction posted for  28335
Comment posted for  28335
Prediction posted for  28336
Comment posted for  28336
